In [1]:
from __future__ import division
from __future__ import print_function
from linear_gae.evaluation import get_roc_score, clustering_latent_space
from linear_gae.kcore import compute_kcore, expand_embedding
from linear_gae.model import *
from linear_gae.optimizer import OptimizerAE, OptimizerVAE
from linear_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time
import networkx as nx
import pickle as pkl
import sys
import pandas as pd

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')   # 添加的，不报错

In [3]:
# Select graph dataset
flags.DEFINE_string('dataset', 'email', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'link_prediction', 'Name of the learning task')

# Model
flags.DEFINE_string('model', 'gcn_ae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner
           product decoder

- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS
             workshop paper, with linear encoder, and inner product decoder

- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3
              of NeurIPS workshop paper, with Gaussian priors, linear encoders
              for mu and sigma, and inner product decoder
 
- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4
               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner
               product decoder
 
- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced
                in section 4 of NeurIPS workshop paper, with Gaussian priors,
                3-layer GCN encoders for mu and sigma, and inner product
                decoder
'''

' Available Models:\n\n- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer\n          GCN encoder and inner product decoder\n\n- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with\n           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner\n           product decoder\n\n- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS\n             workshop paper, with linear encoder, and inner product decoder\n\n- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3\n              of NeurIPS workshop paper, with Gaussian priors, linear encoders\n              for mu and sigma, and inner product decoder\n \n- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4\n               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner\n               product decoder\n \n- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced\n                

In [4]:
# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in encoder')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer(s).')
flags.DEFINE_integer('dimension', 16, 'Dimension of encoder output, i.e. \
                                       embedding dimension')

# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Link Prediction task)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Link Prediction task)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results at each epoch (for \
                                           Link Prediction task)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

# Parameters related to the "degeneracy framework" from IJCAI 2019 paper,
# aiming at scaling-up Graph AE/VAE by training the model only on the k-core
# (smaller) version of the graph, then expanding embedding to remaining nodes
# via simpler and faster heuristics
flags.DEFINE_boolean('kcore', False, 'Whether to run k-core decomposition \
                                      and use the framework. False = model \
                                      will be trained on the entire graph')
flags.DEFINE_integer('k', 2, 'Which k-core to use. Higher k => smaller graphs\
                              and faster (but maybe less accurate) training')
flags.DEFINE_integer('nb_iterations', 10, 'Number of fix point iterations in \
                                           algorithm 2 of IJCAI paper. See \
                                           kcore.py file for details')

# Lists to collect average results
if FLAGS.task == 'link_prediction':
    mean_roc = []
    mean_ap = []

if FLAGS.kcore:
    mean_time_kcore = []
    mean_time_train = []
    mean_time_expand = []
    mean_core_size = []
mean_time = []

In [5]:
def load_data(filename_adj, nodes_numbers):
    raw_edges = pd.read_csv(filename_adj,header=None,sep=' ') - 1
    
    drop_self_loop = raw_edges[raw_edges[0] != raw_edges[1]]
    
    graph_np = np.zeros((nodes_numbers, nodes_numbers))
    
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0], drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1], drop_self_loop.iloc[i,0]]=1
        
    adj = nx.adjacency_matrix(nx.from_numpy_matrix(graph_np))
    
    features = sp.lil_matrix(np.eye(nodes_numbers))
    
    return adj, features

In [6]:
# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")

adj_name = 'datasets/ia-email-univ.mtx'
nodes_number = 1133
adj_init, features_init = load_data(adj_name, nodes_number)

Loading data...


In [7]:
# The entire training+test process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    if FLAGS.task == 'link_prediction' :
        if FLAGS.verbose:
            print("Masking test edges...")
        # Edge Masking for Link Prediction: compute Train/Validation/Test set
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)

    # Start computation of running times
    t_start = time.time()

    # Degeneracy Framework / K-Core Decomposition
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Starting k-core decomposition of the graph")
        # Save adjacency matrix of un-decomposed graph
        # (needed to embed nodes that are not in k-core, after GAE training)
        adj_orig = adj
        # Get the (smaller) adjacency matrix of the k-core subgraph,
        # and the corresponding nodes
        adj, nodes_kcore = compute_kcore(adj, FLAGS.k)
        # Get the (smaller) feature matrix of the nb_core graph
        if FLAGS.features:
            features = features_init[nodes_kcore,:]
        # Flag to compute k-core decomposition's running time
        t_core = time.time()
    elif FLAGS.features:
        features = features_init

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes,
                               features_nonzero)
    elif FLAGS.model == 'deep_gcn_ae':
        # Deep (3-layer GCN) Graph Autoencoder
        model = DeepGCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'deep_gcn_vae':
        # Deep (3-layer GCN) Graph Variational Autoencoder
        model = DeepGCNModelVAE(placeholders, num_features, num_nodes,
                                features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'linear_ae', 'deep_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'linear_vae', 'deep_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")

    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation, for Link Prediction
            if not FLAGS.kcore and FLAGS.validation and FLAGS.task == 'link_prediction':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = get_roc_score(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Flag to compute Graph AE/VAE training time
    t_model = time.time()

    # Compute embedding

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # If k-core is used, only part of the nodes from the original
    # graph are embedded. The remaining ones are projected in the
    # latent space via the expand_embedding heuristic
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Propagation to remaining nodes...")
        # Project remaining nodes in latent space
        emb = expand_embedding(adj_orig, emb, nodes_kcore, FLAGS.nb_iterations)
        # Compute mean running times for K-Core, GAE Train and Propagation steps
        mean_time_expand.append(time.time() - t_model)
        mean_time_train.append(t_model - t_core)
        mean_time_kcore.append(t_core - t_start)
        # Compute mean size of K-Core graph
        # Note: size is fixed if task is node clustering, but will vary if
        # task is link prediction due to edge masking
        mean_core_size.append(len(nodes_kcore))

    # Compute mean total running time
    mean_time.append(time.time() - t_start)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Link Prediction: classification edges/non-edges
    if FLAGS.task == 'link_prediction':
        # Get ROC and AP scores
        ap_score, roc_score = get_roc_score(test_edges, test_edges_false, emb)
        # Report scores
        mean_ap.append(ap_score)
        mean_roc.append(roc_score)

Masking test edges...
Preprocessing and Initializing...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
targets is deprecated, use labels instead

Training...
Epoch: 0001 train_loss= 0.73512 time= 0.16733
Epoch: 0002 train_loss= 0.73476 time= 0.03391
Epoch: 0003 train_loss= 0.73374 time= 0.03591
Epoch: 0004 train_loss= 0.73160 time= 0.03491
Epoch: 0005 train_loss= 0.72782 time= 0.03790
Epoch: 0006 train_loss= 0.72184 time= 0.04720
Epoch: 0007 train_loss= 0.71325 time= 0.04441
Epoch: 0008 train_loss= 0.70209 time= 0.04683
Epoch: 0009 train_loss= 0.68947 time= 0.04508
Epoch: 0010 train_loss= 0.67859 time= 0.04564
Epoch: 0011 train_loss= 0.67481 time= 0.03585
Epoch: 0012 train_loss= 0.67840 time= 0.03690
Epoch: 0013 train_loss= 0.67786 time= 0.03499
Epoch: 0014 train_loss= 0.67105 time= 0.04538
Epoch: 0015 train_loss= 0.66286 time= 0.04484
Epoch: 0016 train_loss= 0.65680 time= 0.03869
Epoch

Epoch: 0132 train_loss= 0.44998 time= 0.05286
Epoch: 0133 train_loss= 0.44927 time= 0.04288
Epoch: 0134 train_loss= 0.44861 time= 0.04089
Epoch: 0135 train_loss= 0.44799 time= 0.03483
Epoch: 0136 train_loss= 0.44742 time= 0.03771
Epoch: 0137 train_loss= 0.44688 time= 0.03582
Epoch: 0138 train_loss= 0.44638 time= 0.03837
Epoch: 0139 train_loss= 0.44591 time= 0.04453
Epoch: 0140 train_loss= 0.44546 time= 0.04708
Epoch: 0141 train_loss= 0.44503 time= 0.04047
Epoch: 0142 train_loss= 0.44460 time= 0.04189
Epoch: 0143 train_loss= 0.44418 time= 0.03812
Epoch: 0144 train_loss= 0.44376 time= 0.03860
Epoch: 0145 train_loss= 0.44336 time= 0.03590
Epoch: 0146 train_loss= 0.44298 time= 0.03794
Epoch: 0147 train_loss= 0.44263 time= 0.04284
Epoch: 0148 train_loss= 0.44230 time= 0.04487
Epoch: 0149 train_loss= 0.44200 time= 0.05386
Epoch: 0150 train_loss= 0.44173 time= 0.04238
Epoch: 0151 train_loss= 0.44148 time= 0.04488
Epoch: 0152 train_loss= 0.44124 time= 0.03772
Epoch: 0153 train_loss= 0.44101 ti

Epoch: 0109 train_loss= 0.47181 time= 0.05023
Epoch: 0110 train_loss= 0.47122 time= 0.03894
Epoch: 0111 train_loss= 0.47067 time= 0.03743
Epoch: 0112 train_loss= 0.47014 time= 0.03631
Epoch: 0113 train_loss= 0.46963 time= 0.03565
Epoch: 0114 train_loss= 0.46911 time= 0.04161
Epoch: 0115 train_loss= 0.46857 time= 0.04444
Epoch: 0116 train_loss= 0.46800 time= 0.04149
Epoch: 0117 train_loss= 0.46740 time= 0.03677
Epoch: 0118 train_loss= 0.46678 time= 0.03594
Epoch: 0119 train_loss= 0.46614 time= 0.03496
Epoch: 0120 train_loss= 0.46550 time= 0.03491
Epoch: 0121 train_loss= 0.46486 time= 0.04588
Epoch: 0122 train_loss= 0.46423 time= 0.04787
Epoch: 0123 train_loss= 0.46360 time= 0.04588
Epoch: 0124 train_loss= 0.46299 time= 0.04488
Epoch: 0125 train_loss= 0.46238 time= 0.04089
Epoch: 0126 train_loss= 0.46178 time= 0.03790
Epoch: 0127 train_loss= 0.46119 time= 0.03666
Epoch: 0128 train_loss= 0.46061 time= 0.03491
Epoch: 0129 train_loss= 0.46004 time= 0.03990
Epoch: 0130 train_loss= 0.45949 ti

Epoch: 0087 train_loss= 0.48666 time= 0.03889
Epoch: 0088 train_loss= 0.48601 time= 0.03872
Epoch: 0089 train_loss= 0.48540 time= 0.04258
Epoch: 0090 train_loss= 0.48478 time= 0.05053
Epoch: 0091 train_loss= 0.48411 time= 0.04740
Epoch: 0092 train_loss= 0.48335 time= 0.03821
Epoch: 0093 train_loss= 0.48251 time= 0.03859
Epoch: 0094 train_loss= 0.48159 time= 0.03945
Epoch: 0095 train_loss= 0.48061 time= 0.03624
Epoch: 0096 train_loss= 0.47957 time= 0.04625
Epoch: 0097 train_loss= 0.47850 time= 0.04728
Epoch: 0098 train_loss= 0.47744 time= 0.03889
Epoch: 0099 train_loss= 0.47640 time= 0.03682
Epoch: 0100 train_loss= 0.47539 time= 0.03484
Epoch: 0101 train_loss= 0.47443 time= 0.03675
Epoch: 0102 train_loss= 0.47351 time= 0.04591
Epoch: 0103 train_loss= 0.47266 time= 0.04426
Epoch: 0104 train_loss= 0.47188 time= 0.04447
Epoch: 0105 train_loss= 0.47117 time= 0.03989
Epoch: 0106 train_loss= 0.47054 time= 0.03989
Epoch: 0107 train_loss= 0.46997 time= 0.03890
Epoch: 0108 train_loss= 0.46945 ti

Epoch: 0065 train_loss= 0.49791 time= 0.03944
Epoch: 0066 train_loss= 0.49615 time= 0.04025
Epoch: 0067 train_loss= 0.49444 time= 0.04255
Epoch: 0068 train_loss= 0.49275 time= 0.04488
Epoch: 0069 train_loss= 0.49113 time= 0.04289
Epoch: 0070 train_loss= 0.48966 time= 0.04588
Epoch: 0071 train_loss= 0.48835 time= 0.04787
Epoch: 0072 train_loss= 0.48722 time= 0.03790
Epoch: 0073 train_loss= 0.48622 time= 0.04089
Epoch: 0074 train_loss= 0.48533 time= 0.03841
Epoch: 0075 train_loss= 0.48449 time= 0.03405
Epoch: 0076 train_loss= 0.48362 time= 0.04341
Epoch: 0077 train_loss= 0.48266 time= 0.04045
Epoch: 0078 train_loss= 0.48157 time= 0.04767
Epoch: 0079 train_loss= 0.48040 time= 0.04139
Epoch: 0080 train_loss= 0.47922 time= 0.04347
Epoch: 0081 train_loss= 0.47806 time= 0.03532
Epoch: 0082 train_loss= 0.47698 time= 0.03536
Epoch: 0083 train_loss= 0.47599 time= 0.03690
Epoch: 0084 train_loss= 0.47510 time= 0.03591
Epoch: 0085 train_loss= 0.47430 time= 0.04687
Epoch: 0086 train_loss= 0.47360 ti

...

Epoch: 0008 train_loss= 0.68998 time= 0.03838
Epoch: 0009 train_loss= 0.67889 time= 0.03890
Epoch: 0010 train_loss= 0.67817 time= 0.04388
Epoch: 0011 train_loss= 0.68283 time= 0.03989
Epoch: 0012 train_loss= 0.67929 time= 0.04089
Epoch: 0013 train_loss= 0.67096 time= 0.03491
Epoch: 0014 train_loss= 0.66411 time= 0.03920
Epoch: 0015 train_loss= 0.66089 time= 0.03660
Epoch: 0016 train_loss= 0.66002 time= 0.03552
Epoch: 0017 train_loss= 0.65937 time= 0.04587
Epoch: 0018 train_loss= 0.65768 time= 0.05186
Epoch: 0019 train_loss= 0.65470 time= 0.04887
Epoch: 0020 train_loss= 0.65078 time= 0.04123
Epoch: 0021 train_loss= 0.64669 time= 0.04306
Epoch: 0022 train_loss= 0.64322 time= 0.03526
Epoch: 0023 train_loss= 0.64077 time= 0.03707
Epoch: 0024 train_loss= 0.63894 time= 0.03601
Epoch: 0025 train_loss= 0.63675 time= 0.03449
Epoch: 0026 train_loss= 0.63350 time= 0.04322
Epoch: 0027 train_loss= 0.62938 time= 0.04457
Epoch: 0028 train_loss= 0.62509 time= 0.04794
Epoch: 0029 train_loss= 0.62116 ti

Epoch: 0190 train_loss= 0.44068 time= 0.04900
Epoch: 0191 train_loss= 0.44041 time= 0.04076
Epoch: 0192 train_loss= 0.44014 time= 0.03790
Epoch: 0193 train_loss= 0.43988 time= 0.03619
Epoch: 0194 train_loss= 0.43962 time= 0.03591
Epoch: 0195 train_loss= 0.43936 time= 0.03690
Epoch: 0196 train_loss= 0.43912 time= 0.03491
Epoch: 0197 train_loss= 0.43887 time= 0.03890
Epoch: 0198 train_loss= 0.43863 time= 0.04389
Epoch: 0199 train_loss= 0.43840 time= 0.04400
Epoch: 0200 train_loss= 0.43816 time= 0.04354
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 0.73514 time= 0.17652
Epoch: 0002 train_loss= 0.73491 time= 0.04756
Epoch: 0003 train_loss= 0.73422 time= 0.04074
Epoch: 0004 train_loss= 0.73268 time= 0.03502
Epoch: 0005 train_loss= 0.72977 time= 0.03617
Epoch: 0006 train_loss= 0.72489 time= 0.03863
Epoch: 0007 train_loss= 0.71747 time= 0.03989
Epoch: 0008 train_loss= 0.70724 time= 0.04623
Epoch: 0009 train_loss= 0.69478 time= 0.0

Epoch: 0167 train_loss= 0.44424 time= 0.03890
Epoch: 0168 train_loss= 0.44394 time= 0.03790
Epoch: 0169 train_loss= 0.44365 time= 0.04238
Epoch: 0170 train_loss= 0.44335 time= 0.04289
Epoch: 0171 train_loss= 0.44304 time= 0.04288
Epoch: 0172 train_loss= 0.44272 time= 0.04488
Epoch: 0173 train_loss= 0.44240 time= 0.04189
Epoch: 0174 train_loss= 0.44205 time= 0.04358
Epoch: 0175 train_loss= 0.44170 time= 0.03590
Epoch: 0176 train_loss= 0.44134 time= 0.03690
Epoch: 0177 train_loss= 0.44098 time= 0.03817
Epoch: 0178 train_loss= 0.44061 time= 0.04461
Epoch: 0179 train_loss= 0.44024 time= 0.05186
Epoch: 0180 train_loss= 0.43988 time= 0.05386
Epoch: 0181 train_loss= 0.43952 time= 0.04189
Epoch: 0182 train_loss= 0.43917 time= 0.04364
Epoch: 0183 train_loss= 0.43883 time= 0.04319
Epoch: 0184 train_loss= 0.43850 time= 0.03949
Epoch: 0185 train_loss= 0.43818 time= 0.03989
Epoch: 0186 train_loss= 0.43788 time= 0.03639
Epoch: 0187 train_loss= 0.43758 time= 0.04488
Epoch: 0188 train_loss= 0.43730 ti

Epoch: 0147 train_loss= 0.45030 time= 0.04367
Epoch: 0148 train_loss= 0.44997 time= 0.04378
Epoch: 0149 train_loss= 0.44965 time= 0.03968
Epoch: 0150 train_loss= 0.44932 time= 0.04488
Epoch: 0151 train_loss= 0.44900 time= 0.04488
Epoch: 0152 train_loss= 0.44867 time= 0.04488
Epoch: 0153 train_loss= 0.44835 time= 0.04787
Epoch: 0154 train_loss= 0.44803 time= 0.04190
Epoch: 0155 train_loss= 0.44772 time= 0.03988
Epoch: 0156 train_loss= 0.44741 time= 0.03674
Epoch: 0157 train_loss= 0.44711 time= 0.03801
Epoch: 0158 train_loss= 0.44682 time= 0.03979
Epoch: 0159 train_loss= 0.44653 time= 0.04151
Epoch: 0160 train_loss= 0.44626 time= 0.04674
Epoch: 0161 train_loss= 0.44599 time= 0.04566
Epoch: 0162 train_loss= 0.44573 time= 0.04488
Epoch: 0163 train_loss= 0.44547 time= 0.04318
Epoch: 0164 train_loss= 0.44521 time= 0.04688
Epoch: 0165 train_loss= 0.44495 time= 0.04189
Epoch: 0166 train_loss= 0.44469 time= 0.04098
Epoch: 0167 train_loss= 0.44442 time= 0.03541
Epoch: 0168 train_loss= 0.44415 ti

Epoch: 0128 train_loss= 0.45885 time= 0.04449
Epoch: 0129 train_loss= 0.45834 time= 0.04011
Epoch: 0130 train_loss= 0.45784 time= 0.04073
Epoch: 0131 train_loss= 0.45735 time= 0.03463
Epoch: 0132 train_loss= 0.45687 time= 0.03742
Epoch: 0133 train_loss= 0.45641 time= 0.04196
Epoch: 0134 train_loss= 0.45597 time= 0.04789
Epoch: 0135 train_loss= 0.45554 time= 0.04825
Epoch: 0136 train_loss= 0.45513 time= 0.04614
Epoch: 0137 train_loss= 0.45472 time= 0.04592
Epoch: 0138 train_loss= 0.45431 time= 0.04370
Epoch: 0139 train_loss= 0.45388 time= 0.04359
Epoch: 0140 train_loss= 0.45344 time= 0.04095
Epoch: 0141 train_loss= 0.45298 time= 0.03881
Epoch: 0142 train_loss= 0.45250 time= 0.03657
Epoch: 0143 train_loss= 0.45200 time= 0.04007
Epoch: 0144 train_loss= 0.45148 time= 0.04848
Epoch: 0145 train_loss= 0.45095 time= 0.04992
Epoch: 0146 train_loss= 0.45041 time= 0.05196
Epoch: 0147 train_loss= 0.44988 time= 0.04918
Epoch: 0148 train_loss= 0.44936 time= 0.04106
Epoch: 0149 train_loss= 0.44884 ti

In [8]:
###### Report Final Results ######

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

if FLAGS.task == 'link_prediction':
    print("AP scores\n", mean_ap)
    print("Mean AP score: ", np.mean(mean_ap),
          "\nStd of AP scores: ", np.std(mean_ap), "\n \n")
    
    print("AUC scores\n", mean_roc)
    print("Mean AUC score: ", np.mean(mean_roc),
          "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

if FLAGS.kcore:
    print("Details on degeneracy framework, with k =", FLAGS.k, ": \n \n")

    print("Running times for k-core decomposition\n", mean_time_kcore)
    print("Mean: ", np.mean(mean_time_kcore),
          "\nStd: ", np.std(mean_time_kcore), "\n \n")

    print("Running times for autoencoder training\n", mean_time_train)
    print("Mean: ", np.mean(mean_time_train),
          "\nStd: ", np.std(mean_time_train), "\n \n")

    print("Running times for propagation\n", mean_time_expand)
    print("Mean: ", np.mean(mean_time_expand),
          "\nStd: ", np.std(mean_time_expand), "\n \n")

    print("Sizes of k-core subgraph\n", mean_core_size)
    print("Mean: ", np.mean(mean_core_size),
          "\nStd: ", np.std(mean_core_size), "\n \n")


Test results for gcn_ae model on email on link_prediction 
 ___________________________________________________

AP scores
 [0.882687123383034, 0.8864427914849348, 0.896048454155328, 0.881520415367957, 0.8955395569707058, 0.8900340394197731, 0.904384558604975, 0.8850908821661818, 0.8755752825528204, 0.9059179383000182]
Mean AP score:  0.8903241042405728 
Std of AP scores:  0.00946955469851858 
 

AUC scores
 [0.8634492046124064, 0.8606514603147883, 0.8740577392475382, 0.8527076845383383, 0.8826563420587493, 0.8544533288443734, 0.8876929551384564, 0.8696237690430099, 0.8585337934517296, 0.8881003282551972]
Mean AUC score:  0.8691926605504587 
Std of AUC scores:  0.012702123962401127 
 

